In [ ]:
from pandas import read_csv
from numpy import arange
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
from statistics import NormalDist

In [ ]:
plt.rcParams.update({
        'font.family': 'serif',
        'font.serif': [],
        'font.size' : 11,
        'text.usetex': True,
        'axes.facecolor':'white',
        'axes.edgecolor': 'lightgrey',
        'grid.color': 'lightgrey'
        })

MEDIUM_SIZE = 14
BIGGER_SIZE = 16
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=12)             # legend fontsize

In [ ]:
PREFIX = "/mnt/harddrive/data/backup/old/FibroLAN-FalconRX"
MEASUREMENTS = ["_IEEE-802.1p_60_120.0percent",
                "_IEEE-802.1p_1518_120.0percent",
                "-BBB_IEEE-802.1p_60_120.0percent",
                "-BBB_IEEE-802.1p_1518_120.0percent"]
PATHS = ["{}{}".format(PREFIX, p) for p in MEASUREMENTS]
BARS = ["minimum",
        "mean",
        "0.999_percentile",
        "maximum",
        "std"]
STREAMS = ["robotic", "audio", "video"]
METRIC = "Latency (ns)"

In [ ]:
dfs = {}
for m in PATHS:
    dfs[m] = read_csv(m + "/csv/statistics.csv", delimiter=";")

dfs_raw = {}
for p in PATHS:
    dfs_raw[p] = {}
    for s in STREAMS:
        dfs_raw[p][s] = read_csv('{}/csv/{}.csv'.format(p, s),
                                 delimiter=",",
                                 usecols=['Latency (ns)'],
                                 dtype=int)

In [ ]:
data = {}
for file, d in dfs.items():
    # select specific rows and columns from csv/dataframes:
    filtered_df = {}
    for s in STREAMS:
        stream_data = []
        row = d.loc[(d['stream_pattern'].eq(s)) & (d['measurement'].eq(METRIC))]
        # select only desired statistical values:
        for b in BARS:
            # add value in µs:
            stream_data.append(row.iloc[0][b] / 1e3)
        filtered_df[s] = stream_data
    data[file] = filtered_df

In [ ]:
# https://stackoverflow.com/a/55270860
def confidence_interval(data, confidence=0.95):
    dist = NormalDist.from_samples(data)
    z = NormalDist().inv_cdf((1 + confidence) / 2.)
    e = dist.stdev * z / ((len(data) - 1) ** .5)
    return e

In [ ]:
cis = {}

for s in STREAMS:
    cis[s] = []

for p in PATHS:
    for s in STREAMS:
        cis[s].append(confidence_interval(dfs_raw[p][s]['Latency (ns)']))

In [ ]:
means = []

for p in PATHS:
    means.append(data[p]['robotic'][1])

In [ ]:
barWidth = 0.20
x = arange(len(BARS))

for s in STREAMS:
    offset = 0
    fig, ax = plt.subplots()
    ax.bar(x + offset * barWidth,
           data[PATHS[0]][s],
           barWidth-0.01,
           log=False,
           hatch="///",
           edgecolor="black")
    offset +=1
    ax.bar(x + offset * barWidth,
           data[PATHS[1]][s],
           barWidth-0.01,
           log=False,
           hatch='\\\\',
           edgecolor="black")
    offset +=1
    ax.bar(x + offset * barWidth,
           data[PATHS[2]][s],
           barWidth-0.01,
           log=False,
           hatch="oo",
           edgecolor="black")
    offset +=1
    ax.bar(x + offset * barWidth,
           data[PATHS[3]][s],
           barWidth-0.01,
           log=False,
           hatch="..",
           edgecolor="black")
    ax.errorbar(x=[1, 1.2, 1.4, 1.6],
                y=means,
                yerr=cis[s],
                ls='none',
                color='black',
                capsize=5)
    plt.ylabel('One-way Delay [µs]')
    x_offset = x + (len(MEASUREMENTS) - 1) * barWidth / 2
    xlabels = ["Min",
               "Mean",
               "P99.9\%",
               "Max",
               "Stdev"]
    ax.tick_params(axis='y')
    plt.xticks(x_offset,
               xlabels,
               ha="center")
    legend = ["64B", "1522B", "Adaptive 64B", "Adaptive 1522B"]
    plt.legend(legend,
               bbox_to_anchor=(0, 1, 1, 0),
               loc="lower left",
               mode="expand",
               ncol=4,
               frameon=True,
               handletextpad=0.3)
    plt.ylim(0,30)
    plt.tight_layout()
    plt.savefig("barplot_{}.pdf".format(s))
    plt.close("all")
    #plt.show()